# Proyecto: Búsqueda en Árbol — *Peg Solitaire* (Come Solo)

Autores: **Juan Sebastian Abarca Hernandez y Juan Antonio Trenado Ballesteros**  
Fecha: 24 de Septiembre de 2025

**Repositorio base (fork de `baile`)**: https://github.com/SebastianAbarca25/baileProyecto1

Este cuaderno documenta la **definición formal del problema**, los **experimentos** con búsqueda **no informada** (BFS, DFS) y **A\***, la **comparación de resultados** y las **conclusiones**.


## Instrucciones rápidas
1. Ejecuta todas las celdas en orden. 
2. Para **probar otras configuraciones** (por ejemplo, tableros alternos o profundidad límite para DFS), modifica los parámetros en la sección de *Experimentos*.
3. Al final hay una celda para **imprimir la secuencia de movimientos** y **reconstruir la solución**.

### Citas y contexto
- Librería base: `baile` — *Baisic Artificial Intelligence Library for Education* (GitHub: `https://github.com/kyriox/baile`).
- Dominio: *Peg Solitaire* (tablero inglés de 33 posiciones, salto ortogonal, se retira la ficha sobre la que se salta; objetivo: una sola ficha, idealmente en el centro).


## 1) Definición formal del problema
**Estado**: representamos el tablero como una **matriz 7×7**, donde:
- `-1` = fuera del tablero (celdas no válidas),
- `0`  = vacío,
- `1`  = hay ficha.

Usamos el tablero inglés estándar (33 casillas válidas). El estado inicial tiene **todas las casillas válidas ocupadas salvo el centro**.

**Operador/sucesor**: un movimiento válido es un **salto ortogonal** (arriba/abajo/izquierda/derecha) de una ficha `1` sobre otra `1` hacia una casilla vacía `0`, retirando la ficha intermedia. Formalmente, para una dirección `d∈{(±1,0),(0,±1)}` si `s[i,j]=1`, `s[i+di,j+dj]=1` y `s[i+2di,j+2dj]=0`, entonces se genera un nuevo estado con `s[i,j]=0`, `s[i+di,j+dj]=0`, `s[i+2di,j+2dj]=1`.

**Función de meta**: `goal(s)` es verdadero si el **número de fichas** en `s` es `1`. (Podemos añadir la variante *meta central* que además exige que la ficha esté en el centro `(3,3)`.)

**Heurística (admisible)**:  
Usamos `h₁(s) = max(0, pegs(s) - 1)`.
Justificación: cada movimiento **siempre elimina exactamente una ficha**, por lo que el **número mínimo de movimientos restantes** es al menos `pegs − 1`. Es una cota inferior y, por tanto, **admisible**.

Adicionalmente incluimos una **heurística informativa no admisible** opcional para análisis comparativo: `h₂(s) = (pegs(s)-1) + λ·dist_min_al_centro(s)`, con `λ>0`, para influir el sesgo hacia el centro (solo para análisis, no garantiza optimalidad).


In [ ]:
# Utilidades de tablero Peg Solitaire (7x7 inglés)
from typing import List, Tuple, Iterable, Optional, Dict, Any

Board = List[List[int]]  # -1 fuera, 0 vacío, 1 ficha
Vec = Tuple[int,int]
Move = Tuple[Tuple[int,int], Tuple[int,int]]  # (origen)->(destino)

DIRS: List[Vec] = [(1,0),(-1,0),(0,1),(0,-1)]

def english_board() -> Board:
    # Plantilla del tablero inglés estándar (7x7 con 33 casillas válidas)
    O, X = -1, 1
    row = lambda a: [1 if c=='X' else (-1 if c=='O' else 0) for c in a]
    # Usamos 'X' como casilla válida inicial con ficha; luego vaciamos el centro
    layout = [
        ['O','O','X','X','X','O','O'],
        ['O','O','X','X','X','O','O'],
        ['X','X','X','X','X','X','X'],
        ['X','X','X','0','X','X','X'],  # marcaremos '0' para vacío inicial
        ['X','X','X','X','X','X','X'],
        ['O','O','X','X','X','O','O'],
        ['O','O','X','X','X','O','O'],
    ]
    board: Board = []
    for r in layout:
        br = []
        for c in r:
            if c=='O': br.append(-1)
            elif c=='0': br.append(0)
            else: br.append(1)
        board.append(br)
    return board

def clone(b: Board) -> Board:
    return [row[:] for row in b]

def in_bounds(b: Board, i:int, j:int) -> bool:
    return 0 <= i < 7 and 0 <= j < 7 and b[i][j] != -1

def count_pegs(b: Board) -> int:
    return sum(1 for i in range(7) for j in range(7) if b[i][j]==1)

def moves(b: Board) -> Iterable[Move]:
    for i in range(7):
        for j in range(7):
            if b[i][j] != 1: continue
            for di,dj in DIRS:
                i1,j1 = i+di, j+dj
                i2,j2 = i+2*di, j+2*dj
                if in_bounds(b,i2,j2) and b[i1][j1]==1 and b[i2][j2]==0:
                    yield ((i,j),(i2,j2))

def apply_move(b: Board, m: Move) -> Board:
    (i,j),(i2,j2) = m
    di,dj = (i2-i)//2, (j2-j)//2
    nb = clone(b)
    nb[i][j] = 0
    nb[i+di][j+dj] = 0
    nb[i2][j2] = 1
    return nb

def is_goal_one(b: Board) -> bool:
    return count_pegs(b) == 1

def is_goal_one_center(b: Board) -> bool:
    return count_pegs(b) == 1 and b[3][3]==1

def h_pegs(b: Board) -> int:
    return max(0, count_pegs(b) - 1)

def dist_to_center_min(b: Board) -> int:
    # distancia de Manhattan mínima de alguna ficha al centro (3,3)
    D = []
    for i in range(7):
        for j in range(7):
            if b[i][j]==1: D.append(abs(i-3)+abs(j-3))
    return min(D) if D else 0

def h_non_admissible(b: Board, lam: float = 0.25) -> float:
    return (count_pegs(b) - 1) + lam * dist_to_center_min(b)

def board_str(b: Board) -> str:
    s=''
    for i in range(7):
        row=[]
        for j in range(7):
            row.append({-1:' ',0:'.',1:'o'}[b[i][j]])
        s += ' '.join(row)+"\n"
    return s

start = english_board()
print('Estado inicial:')
print(board_str(start))


## 2) Experimentos: BFS, DFS y A*



## 3) Reconstrucción de la solución y visualización paso a paso


## 4) Conclusiones y observaciones

